In [38]:
import pandas as pd
import numpy as np
from IPython.display import display
import psycopg2

host = "localhost"
user = "erik"
password = "1234"


In [39]:
def page_data_sum(host,user,password):    
    conn = psycopg2.connect(host = host,user = user , password = password)
    querry = """
    SELECT 
    data_type_code,
    COUNT(*) as counts FROM crawldb.page_data
    GROUP BY data_type_code
    
    """
    df = pd.read_sql(querry,conn)
    return df

def pages_per_site():
    conn = psycopg2.connect(host = host,user = user , password = password)
    querry = """
        SELECT
            a.site_id,
            b.domain,
            COUNT(*) as no_pages
        FROM crawldb.page a
        LEFT JOIN crawldb.site b
            ON a.site_id = b.id
        GROUP BY
            a.site_id,
            b.domain
        ORDER BY no_pages DESC  
    """
    df = pd.read_sql(querry,conn)
    return df

def images_per_page():
    conn = psycopg2.connect(host = host,user = user , password = password)
    querry = """
        SELECT
            a.page_id,
            COUNT(*) as no_images
        FROM crawldb.image a
        LEFT JOIN crawldb.page b
            ON a.page_id = b.id
        GROUP BY
            a.page_id
        ORDER BY no_images DESC 
    """
    df = pd.read_sql(querry,conn)
    return df

def get_links():
    conn = psycopg2.connect(host = host,user = user , password = password)
    querry = """
        SELECT * FROM crawldb.link
    """
    df = pd.read_sql(querry,conn)
    return df

def get_pages():
    conn = psycopg2.connect(host = host,user = user , password = password)
    querry = """
        SELECT id,site_id FROM crawldb.page
    """
    df = pd.read_sql(querry,conn)
    return df

In [43]:
data_sum = page_data_sum(host,user,password)
display(data_sum)

data_type_code  counts
0            DOC      91
1            PPT      14
2            PDF    1235
3           PPTX       4
4           DOCX      10

In [45]:
import plotly.graph_objects as go
# animals=['giraffes', 'orangutans', 'monkeys']

# fig = go.Figure([go.Bar(x=, y=data_sum["counts"])])
# fig.show()


x = data_sum["data_type_code"]
y = data_sum["counts"]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])

fig.show()

In [47]:
pps = pages_per_site()
display(pps)

site_id                      domain  no_pages
0         11    www.mvn.e-prostor.gov.si      1144
1        130           portal.evs.gov.si      1008
2         57       eionet-si.arso.gov.si       866
3         92          eionet.arso.gov.si       818
4        104             www.umar.gov.si       778
..       ...                         ...       ...
178       91       presojaucinkov.gov.si         1
179      192      www.inspire.mop.gov.si         1
180      138       ecarina.carina.gov.si         1
181      180  mirs.arhiv-spletisc.gov.si         1
182      185  uvtp.arhiv-spletisc.gov.si         1

[183 rows x 3 columns]

In [48]:
import plotly.graph_objects as go
# animals=['giraffes', 'orangutans', 'monkeys']

# fig = go.Figure([go.Bar(x=, y=data_sum["counts"])])
# fig.show()


x = pps["domain"]
y = pps["no_pages"]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])

fig.show()

In [56]:
df = images_per_page()
display(df)
print(f"Average images per page: {np.sum(df['no_images'])/len(df)}")
display(df)


page_id  no_images
0         294        139
1        2000         77
2          18         77
3         700         71
4        1504         67
...       ...        ...
4337     6676          1
4338     4483          1
4339     4189          1
4340     4299          1
4341     6333          1

[4342 rows x 2 columns]

Average images per page: 7.468217411331183


page_id  no_images
0         294        139
1        2000         77
2          18         77
3         700         71
4        1504         67
...       ...        ...
4337     6676          1
4338     4483          1
4339     4189          1
4340     4299          1
4341     6333          1

[4342 rows x 2 columns]

In [57]:
x = list(range(1000))
# x = x.astype(str)[:1000]
# print(x)
y = df["no_images"][:1000]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])

fig.show()

In [22]:
import json
links = get_links()
pages = get_pages()
display(links)
display(pages)



from_page  to_page
0             2        5
1             3        7
2             3        8
3             3        9
4             3       10
...         ...      ...
7591       7480     7837
7592       7480     7838
7593       7480     7839
7594       7480     7840
7595       7610     7841

[7596 rows x 2 columns]

id  site_id
0        1        1
1        2        2
2        3        4
3        4        3
4        5        8
...    ...      ...
7596  7837       11
7597  7838       11
7598  7839       11
7599  7840       11
7600  7841       11

[7601 rows x 2 columns]

In [37]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white")

# # set the physics layout of the network
# got_net.barnes_hut()
# got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")

display(links)

sources = links['from_page']
targets = links['to_page']
# weights = got_data['Weight']

edge_data = zip(sources, targets)
# print(edge_data)
for e in edge_data:
    print(e)
    src = e[0]
    dst = e[1]
#     w = e[2]

    got_net.add_node(src, src)
    got_net.add_node(dst, dst)
    got_net.add_edge(src, dst)

# neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
# for node in got_net.nodes:
#     node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
#     node["value"] = len(neighbor_map[node["id"]])

got_net.show("gameofthrones.html")

from_page  to_page
0             2        5
1             3        7
2             3        8
3             3        9
4             3       10
...         ...      ...
7591       7480     7837
7592       7480     7838
7593       7480     7839
7594       7480     7840
7595       7610     7841

[7596 rows x 2 columns]

(2, 5)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 12)
(7, 14)
(3, 15)
(3, 17)
(3, 18)
(10, 19)
(3, 20)
(2, 21)
(12, 22)
(20, 23)
(12, 24)
(14, 25)
(3, 26)
(20, 27)
(20, 28)
(3, 30)
(19, 31)
(2, 32)
(12, 34)
(20, 35)
(18, 36)
(20, 37)
(12, 38)
(3, 39)
(27, 40)
(27, 41)
(3, 42)
(20, 43)
(40, 44)
(27, 46)
(3, 47)
(3, 48)
(2, 49)
(30, 50)
(18, 51)
(20, 52)
(43, 53)
(13, 54)
(18, 55)
(39, 56)
(27, 57)
(27, 58)
(18, 59)
(36, 60)
(20, 61)
(3, 62)
(46, 63)
(20, 64)
(52, 65)
(3, 66)
(12, 67)
(20, 69)
(43, 70)
(2, 71)
(18, 72)
(30, 73)
(18, 74)
(39, 75)
(43, 76)
(46, 77)
(46, 78)
(27, 79)
(52, 80)
(43, 81)
(27, 82)
(43, 83)
(20, 84)
(20, 85)
(20, 86)
(46, 87)
(52, 88)
(43, 89)
(3, 90)
(48, 92)
(43, 93)
(69, 94)
(20, 95)
(3, 96)
(46, 97)
(66, 98)
(12, 99)
(43, 100)
(64, 101)
(2, 102)
(81, 103)
(30, 104)
(18, 105)
(18, 106)
(93, 107)
(27, 108)
(69, 109)
(43, 110)
(81, 111)
(59, 112)
(39, 113)
(12, 114)
(20, 115)
(36, 116)
(95, 117)
(95, 118)
(70, 119)
(12, 120)
(78, 121)
(2, 122)
(18, 123)
(110, 124)
(39, 1

(18, 961)
(2, 962)
(564, 963)
(66, 964)
(652, 965)
(355, 966)
(652, 967)
(535, 968)
(27, 969)
(147, 970)
(147, 971)
(483, 972)
(18, 973)
(652, 974)
(18, 975)
(2, 976)
(519, 977)
(564, 978)
(418, 979)
(730, 980)
(18, 981)
(388, 982)
(139, 983)
(553, 984)
(27, 985)
(139, 987)
(32, 988)
(972, 989)
(908, 990)
(27, 991)
(730, 992)
(96, 993)
(355, 994)
(139, 995)
(981, 996)
(418, 997)
(388, 998)
(32, 999)
(535, 1000)
(980, 1001)
(935, 1002)
(535, 1004)
(652, 1005)
(147, 1006)
(1001, 1007)
(18, 1008)
(581, 1009)
(96, 1010)
(564, 1011)
(996, 1012)
(730, 1013)
(139, 1014)
(993, 1015)
(1010, 1016)
(147, 1017)
(18, 1018)
(1010, 1019)
(27, 1020)
(1010, 1021)
(418, 1022)
(730, 1023)
(388, 1024)
(951, 1025)
(27, 1026)
(399, 1028)
(990, 1029)
(139, 1030)
(399, 1031)
(71, 1032)
(147, 1033)
(535, 1034)
(564, 1035)
(96, 1037)
(1010, 1038)
(1010, 1039)
(611, 1040)
(18, 1041)
(27, 1042)
(652, 1043)
(49, 1044)
(632, 1045)
(747, 1046)
(993, 1047)
(535, 1048)
(102, 1049)
(147, 1050)
(139, 1051)
(652, 1053)
(

(206, 1785)
(579, 1786)
(1139, 1787)
(691, 1788)
(36, 1789)
(264, 1790)
(251, 1791)
(1111, 1792)
(1776, 1793)
(1779, 1794)
(1751, 1795)
(677, 1796)
(1110, 1797)
(697, 1798)
(579, 1799)
(1139, 1800)
(264, 1801)
(1266, 1802)
(691, 1803)
(290, 1804)
(36, 1805)
(273, 1806)
(677, 1807)
(1110, 1808)
(303, 1809)
(1751, 1810)
(579, 1811)
(18, 1812)
(691, 1813)
(264, 1814)
(697, 1815)
(273, 1816)
(1266, 1817)
(1139, 1818)
(677, 1819)
(1110, 1820)
(697, 1821)
(18, 1822)
(1266, 1823)
(579, 1824)
(691, 1825)
(264, 1826)
(273, 1827)
(319, 1828)
(1139, 1829)
(1812, 1830)
(677, 1831)
(1830, 1832)
(1110, 1834)
(697, 1835)
(579, 1836)
(1266, 1837)
(18, 1838)
(691, 1839)
(319, 1840)
(273, 1841)
(264, 1842)
(1139, 1843)
(1812, 1844)
(677, 1845)
(1844, 1846)
(1110, 1847)
(697, 1848)
(18, 1849)
(1266, 1850)
(319, 1851)
(579, 1852)
(273, 1853)
(1139, 1854)
(1812, 1855)
(1826, 1856)
(264, 1857)
(1855, 1858)
(692, 1859)
(719, 1860)
(579, 1861)
(18, 1862)
(1110, 1864)
(273, 1865)
(319, 1866)
(1139, 1867)
(1812

(1642, 2863)
(1959, 2864)
(203, 2865)
(2847, 2866)
(441, 2867)
(273, 2868)
(1335, 2869)
(1299, 2870)
(1642, 2871)
(1959, 2872)
(2864, 2873)
(811, 2874)
(2847, 2875)
(203, 2876)
(1299, 2877)
(441, 2878)
(1335, 2879)
(811, 2880)
(273, 2881)
(1642, 2882)
(2847, 2883)
(203, 2884)
(1959, 2885)
(1299, 2886)
(273, 2887)
(441, 2888)
(1335, 2889)
(1642, 2890)
(2847, 2891)
(203, 2892)
(1959, 2893)
(1335, 2894)
(273, 2895)
(1299, 2896)
(441, 2897)
(1642, 2898)
(203, 2899)
(814, 2900)
(2847, 2901)
(1959, 2902)
(273, 2903)
(1335, 2904)
(1299, 2905)
(814, 2906)
(1642, 2907)
(441, 2908)
(203, 2909)
(1959, 2910)
(2847, 2911)
(1335, 2912)
(814, 2913)
(273, 2914)
(441, 2915)
(1299, 2916)
(1642, 2917)
(203, 2918)
(2847, 2919)
(1959, 2920)
(1335, 2921)
(1299, 2922)
(1959, 2923)
(273, 2924)
(1642, 2925)
(441, 2926)
(814, 2927)
(203, 2928)
(2847, 2929)
(814, 2930)
(1335, 2931)
(1299, 2932)
(273, 2933)
(1959, 2934)
(203, 2935)
(1684, 2936)
(2847, 2937)
(441, 2938)
(814, 2939)
(1335, 2940)
(1299, 2941)
(1959,

(2855, 3666)
(511, 3667)
(3641, 3668)
(967, 3669)
(981, 3670)
(1642, 3671)
(1959, 3672)
(445, 3673)
(2891, 3674)
(1335, 3675)
(967, 3676)
(537, 3677)
(981, 3678)
(1299, 3679)
(445, 3680)
(1959, 3681)
(1335, 3682)
(1642, 3683)
(2855, 3684)
(537, 3685)
(981, 3686)
(967, 3687)
(1299, 3688)
(445, 3689)
(1335, 3690)
(1959, 3691)
(2901, 3692)
(967, 3693)
(1642, 3694)
(981, 3695)
(1299, 3696)
(537, 3697)
(445, 3698)
(1959, 3699)
(1642, 3700)
(1335, 3701)
(2891, 3702)
(537, 3703)
(981, 3704)
(967, 3705)
(1299, 3706)
(1959, 3707)
(1335, 3708)
(1642, 3709)
(967, 3710)
(445, 3711)
(2855, 3712)
(981, 3713)
(537, 3714)
(1335, 3715)
(1299, 3716)
(1959, 3717)
(1642, 3718)
(2891, 3719)
(445, 3720)
(981, 3721)
(967, 3722)
(1642, 3723)
(1335, 3724)
(2901, 3725)
(967, 3726)
(1299, 3727)
(591, 3728)
(1959, 3729)
(445, 3730)
(1008, 3731)
(1335, 3732)
(1642, 3733)
(1299, 3734)
(1959, 3735)
(445, 3736)
(2855, 3737)
(591, 3738)
(1018, 3739)
(967, 3740)
(1335, 3741)
(2891, 3742)
(1642, 3743)
(591, 3744)
(967, 

(1736, 4441)
(1126, 4442)
(2392, 4443)
(1799, 4444)
(2819, 4445)
(556, 4446)
(1735, 4447)
(1968, 4448)
(1126, 4449)
(1724, 4450)
(1799, 4451)
(556, 4453)
(1735, 4454)
(2819, 4455)
(1977, 4456)
(1736, 4457)
(1126, 4458)
(1799, 4459)
(1735, 4461)
(556, 4462)
(1736, 4463)
(2829, 4464)
(1126, 4465)
(1977, 4466)
(1799, 4467)
(2392, 4468)
(1735, 4469)
(556, 4470)
(2819, 4471)
(1988, 4472)
(1126, 4473)
(1799, 4474)
(1711, 4475)
(2392, 4476)
(2819, 4477)
(1735, 4478)
(1988, 4479)
(556, 4480)
(1736, 4481)
(1799, 4482)
(1126, 4483)
(1735, 4484)
(2392, 4485)
(556, 4486)
(2819, 4487)
(1988, 4488)
(1711, 4489)
(1799, 4490)
(1126, 4491)
(1748, 4492)
(556, 4493)
(2392, 4494)
(1996, 4495)
(2819, 4496)
(1736, 4497)
(1811, 4498)
(1126, 4499)
(2819, 4500)
(2392, 4501)
(1812, 4502)
(630, 4503)
(1811, 4504)
(1711, 4505)
(1996, 4506)
(1145, 4507)
(2819, 4508)
(630, 4509)
(1822, 4510)
(2392, 4511)
(2028, 4512)
(1811, 4513)
(1736, 4514)
(1145, 4515)
(2819, 4516)
(630, 4517)
(1849, 4518)
(2052, 4519)
(2392, 45

(3608, 5786)
(2829, 5787)
(3309, 5788)
(5040, 5789)
(3331, 5790)
(3597, 5791)
(1893, 5792)
(1893, 5793)
(5040, 5794)
(2839, 5795)
(3316, 5796)
(3341, 5797)
(3608, 5798)
(2819, 5799)
(1893, 5800)
(5040, 5801)
(3350, 5802)
(3597, 5803)
(3333, 5804)
(3325, 5805)
(2829, 5806)
(3360, 5807)
(5040, 5808)
(3345, 5809)
(3608, 5810)
(1893, 5811)
(2839, 5812)
(3368, 5813)
(5040, 5814)
(3354, 5815)
(3597, 5816)
(1893, 5817)
(2819, 5818)
(5040, 5819)
(3364, 5820)
(3381, 5821)
(1893, 5822)
(3597, 5823)
(5040, 5824)
(3608, 5825)
(3389, 5826)
(2829, 5827)
(3373, 5828)
(1893, 5829)
(3391, 5830)
(5040, 5831)
(3399, 5832)
(2839, 5833)
(3597, 5834)
(1893, 5835)
(2829, 5836)
(3401, 5837)
(5040, 5838)
(1893, 5839)
(3405, 5840)
(3597, 5841)
(5040, 5842)
(3413, 5843)
(2839, 5844)
(3608, 5845)
(3409, 5846)
(1893, 5847)
(3422, 5848)
(5040, 5849)
(3419, 5850)
(2829, 5851)
(1893, 5852)
(3608, 5853)
(2839, 5854)
(5040, 5855)
(3423, 5856)
(3431, 5857)
(3608, 5858)
(1893, 5859)
(2819, 5860)
(1893, 5861)
(3440, 5862)

(2010, 6426)
(4159, 6427)
(2010, 6428)
(4440, 6429)
(3653, 6430)
(5073, 6432)
(3645, 6433)
(5096, 6434)
(4464, 6435)
(2010, 6436)
(4159, 6437)
(5073, 6438)
(5096, 6439)
(4440, 6440)
(4159, 6441)
(3653, 6442)
(2047, 6443)
(5096, 6444)
(4464, 6445)
(4159, 6446)
(2041, 6447)
(3645, 6448)
(4159, 6449)
(6415, 6450)
(4440, 6451)
(5105, 6452)
(3653, 6453)
(4159, 6454)
(2047, 6455)
(3645, 6456)
(4440, 6457)
(5120, 6458)
(2041, 6459)
(4159, 6460)
(5127, 6461)
(4464, 6462)
(3653, 6463)
(6441, 6464)
(4159, 6465)
(5133, 6466)
(2041, 6467)
(3645, 6468)
(4464, 6469)
(4159, 6470)
(4464, 6471)
(2047, 6472)
(3653, 6473)
(5139, 6474)
(2041, 6475)
(4440, 6476)
(4159, 6477)
(5145, 6478)
(3645, 6479)
(2041, 6480)
(4159, 6481)
(4440, 6482)
(5151, 6483)
(3653, 6484)
(2041, 6485)
(4464, 6486)
(4159, 6487)
(5157, 6488)
(3653, 6489)
(2041, 6490)
(4440, 6491)
(3653, 6492)
(5162, 6493)
(4159, 6494)
(4159, 6495)
(4464, 6496)
(2041, 6497)
(3645, 6498)
(5168, 6499)
(6477, 6500)
(4159, 6501)
(4464, 6502)
(3645, 6503)

(6386, 7094)
(6665, 7095)
(4440, 7096)
(4921, 7097)
(4712, 7098)
(6669, 7099)
(6391, 7100)
(4440, 7101)
(4921, 7102)
(4712, 7103)
(6395, 7104)
(4464, 7105)
(6673, 7106)
(4712, 7107)
(4921, 7108)
(6400, 7109)
(4440, 7110)
(6679, 7111)
(4921, 7112)
(4712, 7113)
(6406, 7114)
(4464, 7115)
(6689, 7116)
(4921, 7117)
(4712, 7118)
(4440, 7119)
(6406, 7120)
(6685, 7121)
(4921, 7122)
(4712, 7123)
(6411, 7124)
(4464, 7125)
(6693, 7126)
(4998, 7127)
(4712, 7128)
(6701, 7129)
(4464, 7130)
(6415, 7131)
(4712, 7132)
(4998, 7133)
(4440, 7134)
(6423, 7135)
(6706, 7136)
(4998, 7137)
(6842, 7138)
(4440, 7139)
(6419, 7140)
(6710, 7141)
(4998, 7142)
(7114, 7143)
(6842, 7144)
(7114, 7145)
(7143, 7146)
(4440, 7147)
(6714, 7148)
(7114, 7149)
(6427, 7150)
(4998, 7151)
(6842, 7152)
(7149, 7153)
(6437, 7154)
(4464, 7155)
(6721, 7156)
(4998, 7157)
(6842, 7158)
(7145, 7159)
(6441, 7160)
(6724, 7161)
(4998, 7162)
(7145, 7163)
(4440, 7164)
(6842, 7165)
(6728, 7166)
(4464, 7167)
(4998, 7168)
(6446, 7169)
(6842, 7170)

In [31]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")

display(got_data)

sources = got_data['Source']
targets = got_data['Target']
weights = got_data['Weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

got_net.show("gameofthrones.html")

Source       Target  Weight
0      Aemon        Grenn       5
1      Aemon      Samwell      31
2      Aerys        Jaime      18
3      Aerys       Robert       6
4      Aerys       Tyrion       5
..       ...          ...     ...
347   Walder        Petyr       6
348   Walder       Roslin       6
349   Walton        Jaime      10
350  Ygritte       Qhorin       7
351  Ygritte  Rattleshirt       9

[352 rows x 3 columns]